In [ ]:
!pip install ta
!pip install pandas-ta
!pip install investpy

In [ ]:
import pandas as pd
import numpy as np
import investpy
import seaborn as sns
from copy import copy
import matplotlib.pyplot as plt
import statistics as stats
import math
from ta.volume import MFIIndicator

# Import données #

In [ ]:
df_source = investpy.get_etf_historical_data(etf='Lyxor UCITS NASDAQ-100 Daily Leverage',
                                            country='France',
                                            from_date='01/10/2006',
                                            to_date='02/06/2022')

df_source.drop(columns=['Currency','Exchange'], inplace=True)
df_source.reset_index(inplace=True)
df_source.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [ ]:
df_source.head(1)

,Date,Open,High,Low,Close,Volume
0,2006-10-02,17.63,17.63,17.44,17.44,590


In [ ]:
df_source.tail(1)

,Date,Open,High,Low,Close,Volume
3995,2022-06-02,562.4,569.3,549.0,567.2,4541


## Ajout indicateurs techniques ##

In [ ]:
df = df_source.copy()

In [ ]:
for i in range(5,51):
  s_mfi = MFIIndicator(high=df_source.High, low=df_source.Low, close=df_source.Close, volume=df_source.Volume, window=i).money_flow_index()
  df['MFI_'+str(i)] = round(s_mfi, 2)
  p_max = i

In [ ]:
# Démarrage lorsque tous les MFI sont renseignés
df = df.loc[(p_max-1):]
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head(1)

,Date,Open,High,Low,Close,Volume,MFI_5,MFI_6,MFI_7,MFI_8,...,MFI_41,MFI_42,MFI_43,MFI_44,MFI_45,MFI_46,MFI_47,MFI_48,MFI_49,MFI_50
0,2006-12-08,18.81,19.27,18.81,19.27,850,29.69,25.11,20.03,40.0,...,58.55,60.0,59.23,59.79,60.67,61.3,61.77,61.77,61.73,61.73


In [ ]:
df.shape

(3947, 52)

In [ ]:
df.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
MFI_5     0
MFI_6     0
MFI_7     0
MFI_8     0
MFI_9     0
MFI_10    0
MFI_11    0
MFI_12    0
MFI_13    0
MFI_14    0
MFI_15    0
MFI_16    0
MFI_17    0
MFI_18    0
MFI_19    0
MFI_20    0
MFI_21    0
MFI_22    0
MFI_23    0
MFI_24    0
MFI_25    0
MFI_26    0
MFI_27    0
MFI_28    0
MFI_29    0
MFI_30    0
MFI_31    0
MFI_32    0
MFI_33    0
MFI_34    0
MFI_35    0
MFI_36    0
MFI_37    0
MFI_38    0
MFI_39    0
MFI_40    0
MFI_41    0
MFI_42    0
MFI_43    0
MFI_44    0
MFI_45    0
MFI_46    0
MFI_47    0
MFI_48    0
MFI_49    0
MFI_50    0
dtype: int64

In [ ]:
# Si besoin de filtrer sur une période plutôt que all time
'''
df = df_source.loc[(df_source['Date']>='2017-01-01')]
df.reset_index(drop=True, inplace=True)
df.head()
'''

"\ndf = df_source.loc[(df_source['Date']>='2017-01-01')]\ndf.reset_index(drop=True, inplace=True)\ndf.head()\n"

# Début programme #

## Fonctions ##

In [ ]:
def performance (me, entreeF, sortieV, df_extrait, col):
    '''Fonction de calcul de la performance 
    avec un seuil d'entrée fixe, et sortie variable'''

    # Définition des entrées sorties
    e = entreeF
    s = me + sortieV

    # Ajout de la colonne mfi_lag
    shift = df_extrait[col].shift(1)
    shift[0]=shift[1]
    df_extrait['mfi_lag']=shift

    # Filtre dans l'historique des points de sorties variables
    # Entrées et sorties à l'ouverture J+1
    entries_indexes = df_extrait.index[ (df_extrait[col]>=e) & (df_extrait['mfi_lag']<(e-0.01)) ].to_list()
    entries_indexes = [x+1 for x in entries_indexes]
    # Attention au dépassement d'index, vérifier si denrnière entrée est bien < à la dernière date enregistrée
    if (len(entries_indexes)>0) and (entries_indexes[-1] >= df_extrait.shape[0]):
      entries_indexes.pop(-1)
    df_entries = df_extrait.loc[entries_indexes, :]
    df_entries['move']='achat'
    # Idem pour les sorties
    exits_indexes = df_extrait.index[ (df_extrait[col]<=s) & (df_extrait['mfi_lag']>(s+0.01)) ].to_list()
    exits_indexes = [y+1 for y in exits_indexes]
    if (len(exits_indexes)>0) and (exits_indexes[-1] >= df_extrait.shape[0]):
      exits_indexes.pop(-1)
    df_exits = df_extrait.loc[exits_indexes, :]
    df_exits['move']='vente'

    # Concatenation des achats/ventes et tri par ordre chronologique
    df_perf = pd.concat([df_entries,df_exits])
    df_perf.sort_values(by='Date',inplace=True)
    df_perf.reset_index(drop=True, inplace=True)

    # Suppression des entrées-sorties doubles (type achat-achat-vente, ou achat-vente-vente)
    doubles = []
    for i in range (1,df_perf.shape[0]):
        if df_perf['move'][i] == df_perf['move'][i-1]:
            doubles.append(i)
        else:
            continue
    df_perf_clean = df_perf.drop(index=doubles)
    df_perf_clean.reset_index(drop=True, inplace=True)

    # Suppression de la 1ere ligne si vente, mais après nettoyage des doublons
    if df_perf_clean.empty != True: 
        if df_perf_clean.iloc[0,4] == 'vente':
            df_perf_clean = df_perf_clean.drop(index=0)
            df_perf_clean.reset_index(drop=True, inplace=True)

    # Si dernière ligne est achat non soldé -> insérer la dernière ligne en vente
    # Obtenir toute la dernière ligne du dataframe origienl, puis concat au df_perf_clean = vente au jour J
    if df_perf_clean.empty != True: 
        if df_perf_clean.iloc[-1,4] == 'achat':
            df_last_row = df_extrait.iloc[-1:].copy()
            df_last_row['move']='vente'
            df_perf_clean = pd.concat([df_perf_clean,df_last_row])
            df_perf_clean.reset_index(drop=True, inplace=True)

    # Calcul de la performance
    perfE = [1000]
    j=0
    # S'assurer que le Dataframe a bien 1 entrée 1 sortie, et n'est pas vide (out of bounds)
    if df_perf_clean.shape[0]>=2:
        for i in range (1,df_perf_clean.shape[0],2):
            '''par step 2 pour ne traiter que les lignes impaires correspondant aux lignes de vente'''
            # Correction avec ajustement des frais de transaction
            # Correction entrée à l'ouverture J+1
            var = (df_perf_clean['Open'][i]-df_perf_clean['Open'][i-1])/df_perf_clean['Open'][i-1] -0.011
            varE = perfE[j]*(1+var)
            perfE.append(varE)
            j+=1
    else:
        # Dans le cas où DF vide : on ajoute la valeur d'entrée pour surperf 0
        perfE.append(1000)
    '''Choix de la performance totale en %'''
    overallP = round(((perfE[-1]-perfE[0])/perfE[0])*100,2)
    
    return overallP

In [ ]:
def es_variables (mediane,entreeV,sortieV,df_extrait,col):
    ''' Fonction de calcul de la performance
    seuils d'entrée et de sortie variables dans les plages définies en amont
    avec equilibre à la médiane '''

    matrice = []

    # Définition des entrées sorties
    for eV in entreeV:
        eF = mediane - eV
        ligne = [performance(mediane,eF,sV,df_extrait,col) for sV in sortieV]
        matrice.append(ligne)

    return matrice

In [ ]:
def export_heatmap (matrice,base,pe,ps,titre):
    colonnes = []
    for i in ps:
        nom = "s="+str(base+i)
        colonnes.append(nom)

    ind = []
    for i in pe:
        nom = "e="+str(base-i)
        ind.append(nom)

    df_HM = pd.DataFrame(matrice,columns=colonnes)
    df_HM.set_index(pd.Index(ind), inplace=True)

    fig, ax = plt.subplots(figsize = (30, 15))
    sns.heatmap(df_HM, cmap ='RdYlGn', linewidths = 0.30, annot = True, fmt=".0f")
    ax.set_title(titre)
    chemin = "/content/drive/MyDrive/Colab Notebooks/performancesMFI/NDQ2x-allTime/NDX2x_AT_" + titre + "_mai22_J1.png"
    plt.savefig(chemin)
    # Protection memoire, sinon ouverture de 30 plots
    plt.close(fig)

## Execution ##

In [ ]:
#df = df_source
print("Buy & Hold Performance = {} %".format(round(((df['Close'].iloc[-1] - df['Close'].iloc[0])/df['Close'].iloc[0])*100,2)) )

Buy & Hold Performance = 2843.44 %


In [ ]:
dict_res = {'MFI':[],'Meilleure Perf':[]}

# Pour l'ensemble des colonnes MFI entre 5 et 50
for i in range(5,51):

    # Selection du MFIp
    colonneMFI = "MFI_"+str(i)
    df_extract = df[['Date','Open',colonneMFI]].copy()

    # Obtention de la mediane
    med = round(df[colonneMFI].median())

    # Définition des plages d'entrée et sortie selon min-max de la colonne MFI sélectionnée
    # On selectionne df_stats qui a éliminé les MFI nuls, sinon ceux en lag à l'origine vont définir 0 en entrée min
    entree_min = math.ceil(df[colonneMFI].min())
    sortie_max = math.trunc(df[colonneMFI].max())
    delta_e = med - entree_min
    delta_s = sortie_max - med
    plage_entrees = [num for num in range(5,delta_e)]
    plage_sorties = [num for num in range(10,delta_s)]

    m = es_variables(med, plage_entrees, plage_sorties, df_extract, colonneMFI)
    tableau_perf = np.array(m)

    dict_res["MFI"].append(colonneMFI)
    dict_res["Meilleure Perf"].append(np.max(tableau_perf))

    export_heatmap(tableau_perf, med, plage_entrees, plage_sorties, colonneMFI)

df_res = pd.DataFrame(dict_res)
df_res.sort_values(by="Meilleure Perf", ascending=False)

,MFI,Meilleure Perf
12,MFI_17,5762.89
7,MFI_12,4993.98
9,MFI_14,4945.61
8,MFI_13,4303.98
10,MFI_15,4258.27
5,MFI_10,3970.47
6,MFI_11,3915.40
11,MFI_16,3818.70
32,MFI_37,3369.43
22,MFI_27,3261.37


In [ ]:
df_res.to_excel("/content/drive/MyDrive/Colab Notebooks/performancesMFI/NDQ2x-allTime/NDX2x_04-29.xlsx", sheet_name='all_time')

# Verification de la performance #

## Vérification unitaire ##

In [ ]:
periode = 12
entree = 56
sortie = 98

In [ ]:
df_best = df[['Date','Open','MFI_'+str(periode)]]

In [ ]:
med = round(df_best['MFI_'+str(periode)].median())
print("Mediane : ", med)

Mediane :  54


In [ ]:
for e in range (48,61):
  entree = e
  
  # Ajout de la colonne mfi_lag
  shift = df_best['MFI_'+str(periode)].shift(1)
  shift[0]=shift[1]
  df_best['mfi_lag']=shift

  # Filtre dans l'historique des points de sorties variables
  # Entrées et sorties à l'ouverture J+1
  entries_indexes = df_best.index[ (df_best['MFI_'+str(periode)]>=entree) & (df_best['mfi_lag']<(entree-0.01)) ].to_list()
  entries_indexes = [x+1 for x in entries_indexes]
  # Attention au dépassement d'index, vérifier si denrnière entrée est bien < à la dernière date enregistrée
  if (len(entries_indexes)>0) and (entries_indexes[-1] >= df_best.shape[0]):
    entries_indexes.pop(-1)
  df_entries = df_best.loc[entries_indexes, :]
  df_entries['move']='achat'
  # Idem pour les sorties
  exits_indexes = df_best.index[ (df_best['MFI_'+str(periode)]<=sortie) & (df_best['mfi_lag']>(sortie+0.01)) ].to_list()
  exits_indexes = [y+1 for y in exits_indexes]
  if (len(exits_indexes)>0) and (exits_indexes[-1] >= df_best.shape[0]):
    exits_indexes.pop(-1)
  df_exits = df_best.loc[exits_indexes, :]
  df_exits['move']='vente'

  # Concatenation des achats/ventes et tri par ordre chronologique
  df_perf = pd.concat([df_entries,df_exits])
  df_perf.sort_values(by='Date',inplace=True)
  df_perf.reset_index(drop=True, inplace=True)

  # Suppression des entrées-sorties doubles (type achat-achat-vente, ou achat-vente-vente)
  doubles = []
  for i in range (1,df_perf.shape[0]):
      if df_perf['move'][i] == df_perf['move'][i-1]:
          doubles.append(i)
      else:
          continue
  df_perf_clean = df_perf.drop(index=doubles)
  df_perf_clean.reset_index(drop=True, inplace=True)

  # Suppression de la 1ere ligne si vente, mais après nettoyage des doublons
  if df_perf_clean.empty != True: 
      if df_perf_clean.iloc[0,4] == 'vente':
          df_perf_clean = df_perf_clean.drop(index=0)
          df_perf_clean.reset_index(drop=True, inplace=True)

  # Si dernière ligne est achat non soldé -> insérer la dernière ligne en vente
  # Obtenir toute la dernière ligne du dataframe origienl, puis concat au df_perf_clean = vente au jour J
  if df_perf_clean.empty != True: 
      if df_perf_clean.iloc[-1,4] == 'achat':
          df_last_row = df_best.iloc[-1:].copy()
          df_last_row['move']='vente'
          df_perf_clean = pd.concat([df_perf_clean,df_last_row])
          df_perf_clean.reset_index(drop=True, inplace=True)
  
  nb_trades = df_perf_clean.shape[0]

  # Calcul de la performance
  perfE = [1000]
  j=0
  # S'assurer que le Dataframe a bien 1 entrée 1 sortie, et n'est pas vide (out of bounds)
  if df_perf_clean.shape[0]>=2:
      for i in range (1,df_perf_clean.shape[0],2):
          '''par step 2 pour ne traiter que les lignes impaires correspondant aux lignes de vente'''
          # Correction avec ajustement des frais de transaction
          # Correction entrée à l'ouverture J+1
          var = (df_perf_clean['Open'][i]-df_perf_clean['Open'][i-1])/df_perf_clean['Open'][i-1] -0.011
          varE = perfE[j]*(1+var)
          perfE.append(varE)
          j+=1
  else:
      # Dans le cas où DF vide : on ajoute la valeur d'entrée pour surperf 0
      perfE.append(1000)
  '''Choix de la performance totale en %'''
  overallP = round(((perfE[-1]-perfE[0])/perfE[0])*100,2)

  print("Entrée : {}, Performance : {} pour {} trades".format(e,overallP,nb_trades) )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Entrée : 48, Performance : 5132.86 pour 12 trades
Entrée : 49, Performance : 5282.34 pour 12 trades
Entrée : 50, Performance : 4910.16 pour 12 trades
Entrée : 51, Performance : 4910.16 pour 12 trades
Entrée : 52, Performance : 5073.99 pour 12 trades
Entrée : 53, Performance : 5073.99 pour 12 trades
Entrée : 54, Performance : 4955.1 pour 12 trades
Entrée : 55, Performance : 5564.61 pour 12 trades
Entrée : 56, Performance : 5363.27 pour 12 trades
Entrée : 57, Performance : 5363.27 pour 12 trades
Entrée : 58, Performance : 5363.27 pour 12 trades
Entrée : 59, Performance : 4759.04 pour 12 trades
Entrée : 60, Performance : 4868.08 pour 12 trades


Idem avec ref 17 38 87

In [ ]:
periode = 17
sortie = 87

df_best = df[['Date','Open','MFI_'+str(periode)]]

med = round(df_best['MFI_'+str(periode)].median())
print("Mediane : ", med)

Mediane :  54


In [ ]:
for e in range (37,40):
  entree = e
  
  # Ajout de la colonne mfi_lag
  shift = df_best['MFI_'+str(periode)].shift(1)
  shift[0]=shift[1]
  df_best['mfi_lag']=shift

  # Filtre dans l'historique des points de sorties variables
  # Entrées et sorties à l'ouverture J+1
  entries_indexes = df_best.index[ (df_best['MFI_'+str(periode)]>=entree) & (df_best['mfi_lag']<(entree-0.01)) ].to_list()
  entries_indexes = [x+1 for x in entries_indexes]
  # Attention au dépassement d'index, vérifier si denrnière entrée est bien < à la dernière date enregistrée
  if (len(entries_indexes)>0) and (entries_indexes[-1] >= df_best.shape[0]):
    entries_indexes.pop(-1)
  df_entries = df_best.loc[entries_indexes, :]
  df_entries['move']='achat'
  # Idem pour les sorties
  exits_indexes = df_best.index[ (df_best['MFI_'+str(periode)]<=sortie) & (df_best['mfi_lag']>(sortie+0.01)) ].to_list()
  exits_indexes = [y+1 for y in exits_indexes]
  if (len(exits_indexes)>0) and (exits_indexes[-1] >= df_best.shape[0]):
    exits_indexes.pop(-1)
  df_exits = df_best.loc[exits_indexes, :]
  df_exits['move']='vente'

  # Concatenation des achats/ventes et tri par ordre chronologique
  df_perf = pd.concat([df_entries,df_exits])
  df_perf.sort_values(by='Date',inplace=True)
  df_perf.reset_index(drop=True, inplace=True)

  # Suppression des entrées-sorties doubles (type achat-achat-vente, ou achat-vente-vente)
  doubles = []
  for i in range (1,df_perf.shape[0]):
      if df_perf['move'][i] == df_perf['move'][i-1]:
          doubles.append(i)
      else:
          continue
  df_perf_clean = df_perf.drop(index=doubles)
  df_perf_clean.reset_index(drop=True, inplace=True)

  # Suppression de la 1ere ligne si vente, mais après nettoyage des doublons
  if df_perf_clean.empty != True: 
      if df_perf_clean.iloc[0,4] == 'vente':
          df_perf_clean = df_perf_clean.drop(index=0)
          df_perf_clean.reset_index(drop=True, inplace=True)

  # Si dernière ligne est achat non soldé -> insérer la dernière ligne en vente
  # Obtenir toute la dernière ligne du dataframe origienl, puis concat au df_perf_clean = vente au jour J
  if df_perf_clean.empty != True: 
      if df_perf_clean.iloc[-1,4] == 'achat':
          df_last_row = df_best.iloc[-1:].copy()
          df_last_row['move']='vente'
          df_perf_clean = pd.concat([df_perf_clean,df_last_row])
          df_perf_clean.reset_index(drop=True, inplace=True)
  
  nb_trades = df_perf_clean.shape[0]

  # Calcul de la performance
  perfE = [1000]
  j=0
  # S'assurer que le Dataframe a bien 1 entrée 1 sortie, et n'est pas vide (out of bounds)
  if df_perf_clean.shape[0]>=2:
      for i in range (1,df_perf_clean.shape[0],2):
          '''par step 2 pour ne traiter que les lignes impaires correspondant aux lignes de vente'''
          # Correction avec ajustement des frais de transaction
          # Correction entrée à l'ouverture J+1
          var = (df_perf_clean['Open'][i]-df_perf_clean['Open'][i-1])/df_perf_clean['Open'][i-1] -0.011
          varE = perfE[j]*(1+var)
          perfE.append(varE)
          j+=1
  else:
      # Dans le cas où DF vide : on ajoute la valeur d'entrée pour surperf 0
      perfE.append(1000)
  '''Choix de la performance totale en %'''
  overallP = round(((perfE[-1]-perfE[0])/perfE[0])*100,2)

  print("Entrée : {}, Performance : {} pour {} trades".format(e,overallP,nb_trades) )

Entrée : 37, Performance : 5611.54 pour 32 trades
Entrée : 38, Performance : 6099.35 pour 32 trades
Entrée : 39, Performance : 5857.24 pour 32 trades


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
